In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

#read in weather data
weather_df = pd.read_csv('../WeatherPy/CityWeatherData.csv')

#Filter columns to be used in weather dataframe
cols = ["City", "Clouds", "Country", "Date", "Humidity", "Lat", "Lon", "Max Temp", "Wind Speed"]
weather_df = weather_df[cols]

#configure gmaps
gmaps.configure(api_key=g_key)


#create coordinates 
coordinates = []
index = 0
locations = weather_df[["Lat", "Lon"]].astype(float)

humidity = weather_df["Humidity"].astype(float)

fig = gmaps.figure()

#create heatmap to display humidity across globe
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

#add heatmap layer
fig.add_layer(heat_layer)

#display heatmap
fig

Figure(layout=FigureLayout(height='420px'))

In [2]:
weather_df.head()

,City,Clouds,Country,Date,Humidity,Lat,Lon,Max Temp,Wind Speed
0,Avarua,32,CK,1578070725,100,-21.21,-159.78,293.15,0.50
1,Ushuaia,40,AR,1578070610,43,-54.81,-68.31,286.15,4.10
2,Mar del Plata,79,AR,1578070324,20,-46.43,-67.52,303.89,18.06
3,Broome,34,GB,1578070613,86,52.47,1.45,279.26,3.10
4,Samarai,86,PG,1578070725,89,-10.62,150.67,297.91,0.86


In [3]:
#convert temperature column to Farenheit
weather_df["Max Temp"] = ( weather_df["Max Temp"] - 273.15 ) * 9 / 5 + 32

In [5]:
#Filter weather dataframe for ideal weather conditions
weather_df = weather_df[weather_df["Max Temp"] < 80]
weather_df = weather_df[weather_df["Max Temp"] > 70]
weather_df = weather_df[weather_df["Wind Speed"] < 10]
weather_df = weather_df[weather_df["Clouds"] == 0]

In [6]:
weather_df

,City,Clouds,Country,Date,Humidity,Lat,Lon,Max Temp,Wind Speed
81,Hermanus,0,ZA,1578070795,71,-34.42,19.24,71.006,0.45
138,Nouadhibou,0,MR,1578070859,19,20.93,-17.03,75.200,4.60
148,Sydney,0,AU,1578070811,83,-33.85,151.22,71.996,1.00
192,Sur,0,OM,1578070927,63,22.57,59.53,70.556,3.90
218,Trincomalee,0,LK,1578070989,94,8.58,81.23,75.362,1.01
229,Fayaoue,0,NC,1578070990,77,-20.65,166.53,75.830,8.66
242,La Rioja,0,AR,1578070991,37,-29.41,-66.86,79.682,3.37
262,Necochea,0,AR,1578070992,71,-38.55,-58.74,71.996,8.15
387,Port Macquarie,0,AU,1578070915,83,-31.43,152.91,71.006,3.60
398,Muscat,0,OM,1578071185,57,23.60,58.55,75.200,0.38


In [59]:
#create url structure for google api calls
base_url = 'https://maps.googleapis.com/maps/api/place/findplacefromtext/json?input=hotel&inputtype=textquery&fields=name,geometry'
location_bias  = '&locationbias=circle:2000@'

#make sure weather dataframe has latitude and longitude as floats so we can create numpy array
locations = weather_df[["Lat", "Lon"]].astype(float)
locations_arr = np.array(locations)

#set up key parameter
key = '&key=' + g_key
hotels = []

#for each location in locations_arr, do a google api call to get json data and add to hotels list
for location in locations_arr:
    response = requests.get(base_url  + location_bias + str(location[0]) + "," + str(location[1]) + key)
    hotels.append(response.json())
    
hotels

-34.42
20.93
-33.85
22.57
8.58
-20.65
-29.41
-38.55
-31.43
23.6
24.63


[{'candidates': [{'geometry': {'location': {'lat': -34.418038,
      'lng': 19.24481},
     'viewport': {'northeast': {'lat': -34.41670937010728,
       'lng': 19.24612107989272},
      'southwest': {'lat': -34.41940902989273, 'lng': 19.24342142010727}}},
    'name': 'The Marine Hermanus'}],
  'status': 'OK'},
 {'candidates': [{'geometry': {'location': {'lat': 20.9213625,
      'lng': -17.0426394},
     'viewport': {'northeast': {'lat': 20.92287727989272,
       'lng': -17.04124047010728},
      'southwest': {'lat': 20.92017762010728, 'lng': -17.04394012989272}}},
    'name': 'HOtel Tasiast'}],
  'status': 'OK'},
 {'candidates': [{'geometry': {'location': {'lat': -33.8414803,
      'lng': 151.208466},
     'viewport': {'northeast': {'lat': -33.84002492010728,
       'lng': 151.2098497298927},
      'southwest': {'lat': -33.84272457989272, 'lng': 151.2071500701073}}},
    'name': 'View Sydney (Formerly North Sydney Harbourview)'}],
  'status': 'OK'},
 {'candidates': [{'geometry': {'loca

In [92]:

hotels[0]

#create list to store hotel locations
hotels_locations = []
indexes = weather_df.index
i =0

#add locations by referencing json data
for hotel in hotels:
    hotels_locations.append({"Index": indexes[i],"Name": hotel['candidates'][0]['name'], "Lat": hotel['candidates'][0]['geometry']['location']['lat'], "Lon": hotel['candidates'][0]['geometry']['location']['lng']})
    i+=1

In [93]:
#confirm data came over correctly
hotels_locations

[{'Index': 81,
  'Name': 'The Marine Hermanus',
  'Lat': -34.418038,
  'Lon': 19.24481},
 {'Index': 138,
  'Name': 'HOtel Tasiast',
  'Lat': 20.9213625,
  'Lon': -17.0426394},
 {'Index': 148,
  'Name': 'View Sydney (Formerly North Sydney Harbourview)',
  'Lat': -33.8414803,
  'Lon': 151.208466},
 {'Index': 192, 'Name': 'Sur Hotel', 'Lat': 22.56835, 'Lon': 59.526532},
 {'Index': 218,
  'Name': 'JKAB Beach Resort',
  'Lat': 8.5960111,
  'Lon': 81.22254099999999},
 {'Index': 229,
  'Name': "Hôtel Paradis d'Ouvéa, Nouvelle Calédonie",
  'Lat': -20.705651,
  'Lon': 166.4632193},
 {'Index': 242, 'Name': 'Plaza Hotel', 'Lat': -29.4133513, 'Lon': -66.8552472},
 {'Index': 262,
  'Name': 'Hotel Ñikén Spa & Business Center',
  'Lat': -38.5812144,
  'Lon': -58.7331016},
 {'Index': 387,
  'Name': 'Sails Port Macquarie by Rydges',
  'Lat': -31.4270974,
  'Lon': 152.8987813},
 {'Index': 398,
  'Name': 'Shangri-La Al Husn Resort & Spa',
  'Lat': 23.5525123,
  'Lon': 58.6601444},
 {'Index': 452,
  'Nam

In [94]:
#convert hotel locations to dataframe so we can join with weather dataframe
hotels_df = pd.DataFrame(hotels_locations)
hotels_df = hotels_df.set_index('Index')

In [104]:

hotels_df = hotels_df[["Name", "Lat", "Lon"]]
hotels_df

#join weather dataframe with hotel dataframe
weather_df = weather_df.join(hotels_df, lsuffix="_w", rsuffix="_h")

In [105]:
#print dataframe to review results
weather_df

,City,Clouds,Country,Date,Humidity,Lat_w,Lon_w,Max Temp,Wind Speed,Name,Lat_h,Lon_h
81,Hermanus,0,ZA,1578070795,71,-34.42,19.24,71.006,0.45,The Marine Hermanus,-34.418038,19.244810
138,Nouadhibou,0,MR,1578070859,19,20.93,-17.03,75.200,4.60,HOtel Tasiast,20.921363,-17.042639
148,Sydney,0,AU,1578070811,83,-33.85,151.22,71.996,1.00,View Sydney (Formerly North Sydney Harbourview),-33.841480,151.208466
192,Sur,0,OM,1578070927,63,22.57,59.53,70.556,3.90,Sur Hotel,22.568350,59.526532
218,Trincomalee,0,LK,1578070989,94,8.58,81.23,75.362,1.01,JKAB Beach Resort,8.596011,81.222541
229,Fayaoue,0,NC,1578070990,77,-20.65,166.53,75.830,8.66,"Hôtel Paradis d'Ouvéa, Nouvelle Calédonie",-20.705651,166.463219
242,La Rioja,0,AR,1578070991,37,-29.41,-66.86,79.682,3.37,Plaza Hotel,-29.413351,-66.855247
262,Necochea,0,AR,1578070992,71,-38.55,-58.74,71.996,8.15,Hotel Ñikén Spa & Business Center,-38.581214,-58.733102
387,Port Macquarie,0,AU,1578070915,83,-31.43,152.91,71.006,3.60,Sails Port Macquarie by Rydges,-31.427097,152.898781
398,Muscat,0,OM,1578071185,57,23.60,58.55,75.200,0.38,Shangri-La Al Husn Resort & Spa,23.552512,58.660144


In [111]:

# NOTE: Do not change any of the code in this cell
#create narrowed dataframe so we can use just relevant info
narrowed_city_df = weather_df[["Name", "City", "Country"]]
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in narrowed_city_df.iterrows()]
locations = hotels_df[["Lat", "Lon"]]

#add symbol layer to gmap
symbols = gmaps.symbol_layer(locations, fill_color='red', stroke_color='red')
fig.add_layer(symbols)
symbol_layer = gmaps.symbol_layer(locations, info_box_content=hotel_info, fill_color="red", stroke_color="red")

In [112]:
fig.add_layer(symbol_layer)

In [113]:
fig

Figure(layout=FigureLayout(height='420px'))